<a href="https://colab.research.google.com/github/dileepkumarg-sa/COVID_19-Data-Analysis/blob/master/2_DL_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from numpy.random import seed
#from tensorflow import set_random_seed
import tensorflow as tf
# from tensorflow.python.keras import backend as K
from tensorflow.python.keras.backend import get_session
from tensorflow import keras
import random as rn
import os
from nltk import tokenize 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Get the file
data_path          = '/content/drive/MyDrive/AOBD/data'
codes_path         = 'AOBD'
cv_models_path     = '/content/drive/MyDrive/AOBD/cv_models/cv_models_toxic'
models_path        = 'models/models_toxic'

In [3]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/AOBD')

In [4]:
if tf.config.list_physical_devices('GPU'):
  import models
else:
  import models_no_gpu as models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
# Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled   = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

train['mal']    = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [6]:
max_features = 40000
maxlen       = 400
dropout_rate = 0.25
#rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150


max_sen_len    = 100
max_sent_amount = 4

#seed(rs)
#tf.random.set_seed(rs)
#rn.seed(rs)

#os.environ['PYTHONHASHSEED']=str(rs)

## Deep Learning Models

### 1. CNN

In [ ]:
models.train_model(X=X, y=y, mtype = 'CNN', cv=True, nfolds=5, epochs=1, cv_models_path=cv_models_path, train=True, rs=rs)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
124/124 [==============================] - 48s 388ms/step
 0.3 fold 0 precision 0.84 recall 0.747 fscore 0.791
fold 0 average precision 0.876
fold 0 roc auc 0.973
 fold 1
Fitting
124/124 [==============================] - 50s 401ms/step
 0.3 fold 1 precision 0.866 recall 0.708 fscore 0.779
fold 1 average precision 0.871
fold 1 roc auc 0.97
 fold 2
Fitting
124/124 [==============================] - 50s 401ms/step
 0.3 fold 2 precision 0.787 recall 0.783 fscore 0.785
fold 2 average precision 0.875
fold 2 roc auc 0.972
 fold 3
Fitting
124/124 [==============================] - 50s 400ms/step
 0.3 fold 3 precision 0.64 recall 0.811 fscore 0.716
fold 3 average precision 0.809
fold 3 roc auc 0.965
 fold 4
Fitting
124/124 [==============================] - 50s 400ms/step
 0.3 fold 4 precision 0.557 recall 0.858 fscore 0.675
fold 4 average precision 0.771
fold 4 roc auc 0.963
PR-C 0.84
ROC AUC 0.969
FScore 0.749
PR-C std 0.043
ROC AUC std 0.004
FScore std 0.046


In [ ]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'CNN', cv=False, nfolds=5, epochs=2, cv_models_path=cv_models_path, train=True, rs=rs)

Fitting
Epoch 1/2
620/620 [==============================] - 957s 2s/step - loss: 0.2102 - acc: 0.9226
Epoch 2/2
254/254 [==============================] - 102s 403ms/step
_________________________________
PR-C is 0.848
_________________________________

_________________________________
ROC AUC is 0.97
_________________________________
_________________________________
FScore is 0.763
_________________________________



### 2. LSTM

In [ ]:
models.train_model(X=X, y=y, mtype = 'LSTM', cv=True, nfolds=5, epochs=2, cv_models_path=cv_models_path, train=True)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/2
496/496 [==============================] - 1117s 2s/step - loss: 0.2536 - acc: 0.9135
Epoch 2/2
124/124 [==============================] - 85s 686ms/step
 0.3 fold 0 precision 0.888 recall 0.717 fscore 0.793
fold 0 average precision 0.881
fold 0 roc auc 0.968
 fold 1
Fitting
Epoch 1/2
496/496 [==============================] - 1111s 2s/step - loss: 0.2763 - acc: 0.9067
Epoch 2/2
124/124 [==============================] - 86s 689ms/step
 0.3 fold 1 precision 0.914 recall 0.622 fscore 0.74
fold 1 average precision 0.853
fold 1 roc auc 0.957
 fold 2
Fitting
Epoch 1/2
496/496 [==============================] - 1102s 2s/step - loss: 0.2650 - acc: 0.9092
Epoch 2/2
124/124 [==============================] - 86s 688ms/step
 0.3 fold 2 precision 0.855 recall 0.727 fscore 0.786
fold 2 average precision 0.863
fold 2 roc auc 0.958
 fold 3
Fitting
Epoch 1/2
496/496 [==============================] - 1103s 2s/step - loss: 0.2491 - acc: 0.9151
Epoch 2/2
124/124 [============

In [ ]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'LSTM', cv=False, nfolds=5, epochs=2, cv_models_path=cv_models_path, train=True, rs=rs)

Fitting
Epoch 1/2
620/620 [==============================] - 1369s 2s/step - loss: 0.2670 - acc: 0.9097
Epoch 2/2
254/254 [==============================] - 175s 687ms/step
_________________________________
PR-C is 0.834
_________________________________

_________________________________
ROC AUC is 0.964
_________________________________
_________________________________
FScore is 0.744
_________________________________



### 3. BLSTM

In [ ]:
models.train_model(X=X, y=y, mtype = 'BLSTM', cv=True, nfolds=5, epochs=2, cv_models_path=cv_models_path, train=True, rs=rs)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/2
496/496 [==============================] - 2140s 4s/step - loss: 0.2452 - acc: 0.9184
Epoch 2/2
124/124 [==============================] - 165s 1s/step
 0.3 fold 0 precision 0.86 recall 0.632 fscore 0.729
fold 0 average precision 0.811
fold 0 roc auc 0.942
 fold 1
Fitting
Epoch 1/2
496/496 [==============================] - 2403s 5s/step - loss: 0.2849 - acc: 0.8993
Epoch 2/2
124/124 [==============================] - 168s 1s/step
 0.3 fold 1 precision 0.928 recall 0.643 fscore 0.76
fold 1 average precision 0.87
fold 1 roc auc 0.966
 fold 2
Fitting
Epoch 1/2
496/496 [==============================] - 2538s 5s/step - loss: 0.2609 - acc: 0.9062
Epoch 2/2
124/124 [==============================] - 169s 1s/step
 0.3 fold 2 precision 0.932 recall 0.645 fscore 0.762
fold 2 average precision 0.852
fold 2 roc auc 0.953
 fold 3
Fitting
Epoch 1/2
 76/496 [===>..........................] - ETA: 40:31 - loss: 0.3826 - acc: 0.8440

In [ ]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BLSTM', cv=False, nfolds=5, epochs=2, cv_models_path=cv_models_path, train=True)

Fitting
Epoch 1/2
620/620 [==============================] - 2989s 5s/step - loss: 0.2452 - acc: 0.9139
Epoch 2/2
254/254 [==============================] - 364s 1s/step
_________________________________
PR-C is 0.849
_________________________________

_________________________________
ROC AUC is 0.969
_________________________________
_________________________________
FScore is 0.767
_________________________________



# 4. GRU

In [ ]:
models.train_model(X=X, y=y, mtype = 'GRU', cv=True, nfolds=5, epochs=2, cv_models_path=cv_models_path, train=True, rs=rs)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/2
496/496 [==============================] - 975s 2s/step - loss: 0.2526 - acc: 0.9181
Epoch 2/2
124/124 [==============================] - 79s 638ms/step
 0.3 fold 0 precision 0.833 recall 0.747 fscore 0.787
fold 0 average precision 0.873
fold 0 roc auc 0.97
 fold 1
Fitting
Epoch 1/2
496/496 [==============================] - 973s 2s/step - loss: 0.2480 - acc: 0.9189
Epoch 2/2
124/124 [==============================] - 79s 634ms/step
 0.3 fold 1 precision 0.835 recall 0.765 fscore 0.799
fold 1 average precision 0.883
fold 1 roc auc 0.972
 fold 2
Fitting
Epoch 1/2
496/496 [==============================] - 963s 2s/step - loss: 0.2447 - acc: 0.9200
Epoch 2/2
124/124 [==============================] - 79s 636ms/step
 0.3 fold 2 precision 0.798 recall 0.809 fscore 0.803
fold 2 average precision 0.888
fold 2 roc auc 0.974
 fold 3
Fitting
Epoch 1/2
496/496 [==============================] - 963s 2s/step - loss: 0.2507 - acc: 0.9190
Epoch 2/2
124/124 [================

In [ ]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'GRU', cv=False, nfolds=5, epochs=2, cv_models_path=cv_models_path, train=True, rs=rs)

Fitting
Epoch 1/2
620/620 [==============================] - 1256s 2s/step - loss: 0.2302 - acc: 0.9256
Epoch 2/2
254/254 [==============================] - 166s 654ms/step
_________________________________
PR-C is 0.85
_________________________________

_________________________________
ROC AUC is 0.969
_________________________________
_________________________________
FScore is 0.76
_________________________________



# 5. BGRU

In [ ]:
models.train_model(X=X, y=y, mtype = 'BGRU', cv=True, nfolds=5, epochs=2, cv_models_path=cv_models_path, train=True, rs=rs)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


 fold 0
Fitting
Epoch 1/2
496/496 [==============================] - 2044s 4s/step - loss: 0.2441 - acc: 0.9176
Epoch 2/2
124/124 [==============================] - 149s 1s/step
 0.3 fold 0 precision 0.804 recall 0.786 fscore 0.795
fold 0 average precision 0.88
fold 0 roc auc 0.97
 fold 1
Fitting
Epoch 1/2
496/496 [==============================] - 2000s 4s/step - loss: 0.2398 - acc: 0.9146
Epoch 2/2
124/124 [==============================] - 148s 1s/step
 0.3 fold 1 precision 0.778 recall 0.804 fscore 0.791
fold 1 average precision 0.879
fold 1 roc auc 0.969
 fold 2
Fitting
Epoch 1/2
496/496 [==============================] - 2009s 4s/step - loss: 0.2464 - acc: 0.9225
Epoch 2/2
124/124 [==============================] - 150s 1s/step
 0.3 fold 2 precision 0.877 recall 0.736 fscore 0.8
fold 2 average precision 0.885
fold 2 roc auc 0.972
 fold 3
Fitting
Epoch 1/2
496/496 [==============================] - 2007s 4s/step - loss: 0.2428 - acc: 0.9175
Epoch 2/2
124/124 [=====================

In [ ]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BGRU', cv=False, nfolds=5, epochs=2, cv_models_path=cv_models_path, train=True, rs=rs)

Fitting
Epoch 1/2
620/620 [==============================] - 2555s 4s/step - loss: 0.2289 - acc: 0.9239
Epoch 2/2
254/254 [==============================] - 308s 1s/step
_________________________________
PR-C is 0.852
_________________________________

_________________________________
ROC AUC is 0.97
_________________________________
_________________________________
FScore is 0.761
_________________________________

